In [ ]:
# basic calculations
import pandas as pd
import numpy as np
import random

# web scraping
from IPython.display import clear_output
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import requests
import time
import json

# data cleaning
import re
from fuzzywuzzy import fuzz
import nltk

### PDF Data

In [ ]:
# get links to all departments' course site
departments = ["mgmt","oid","fnce","statistics","accounting","lgst","hcmg","real-estate"]
link = []
for i in departments:
    link_temp = "https://" + i + ".wharton.upenn.edu/programs/undergraduate/course-descriptions/"
    link.append(link_temp)
    
link.append("https://marketing.wharton.upenn.edu/programs/undergraduate/undergradcourse/")
link.append('https://bepp.wharton.upenn.edu/programs/undergraduate/course-information/course-descriptions/')

In [ ]:
# get the links to the courses
courses = []
for i in link:
    r = requests.get(i)
    soup = BeautifulSoup(r.text, 'html.parser')
    for j in soup.find_all("a"): 
        link_small = j.get("href")
        if link_small.find("syllabi") > 0 and link_small.find("apps") > 0:
            courses.append(link_small)

In [ ]:
# get the syllabus link
syllabus = []
missed = []

for i in courses:
    
    try:
        s = requests.get(i)
        soup = BeautifulSoup(s.text, 'html.parser')
        for i in soup.find_all("a"):
            time.sleep(random.randint(0,4))
            temp_syl = i.get("href")
            if temp_syl.find("syllabi") > 0 and temp_syl.find("2") > 0:
                syllabus.append("https://apps.wharton.upenn.edu" + temp_syl)
                
    except KeyboardInterrupt:
        break
        
    except:
        missed.append(i)
        continue

In [ ]:
# get the pdf syllabus link
pdf_syllabus = []
missed_pdf = []

for i in syllabus:
    try: 
        time.sleep(random.randint(0,4))
        z = requests.get(i)
        soup = BeautifulSoup(z.text, 'html.parser')
        for i in soup.find_all("embed"):
            pdf_syllabus.append(i.get("src"))
    
    except KeyboardInterrupt:
        break
        
    except:
        missed_pdf.append(i)
        continue

In [ ]:
# write files

with open('pdf_syllabus.txt', 'w') as filehandle:  
    for listitem in pdf_syllabus:
        filehandle.write('%s\n' % listitem)


In [ ]:
# read files

f = open('pdf_syllabus_ex_bepp.txt', 'r')
pdf_syllabi = f.readlines()
f.close()


In [ ]:
# get rid of \n

pdf_syllabus = []
for i in range(len(pdf_syllabi)):
    pdf_syllabus.append(pdf_syllabi[i].strip('\n'))


In [ ]:
# read pdf to txt
from urllib.request import urlopen
from urllib.request import Request
from PyPDF2 import PdfFileWriter, PdfFileReader
from io import StringIO
from io import BytesIO
import requests, PyPDF2, io

def pdf_to_string(url):
    listi = []
    response = requests.get(url)
    with io.BytesIO(response.content) as open_pdf_file:
        read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
        number_of_pages = read_pdf.getNumPages()
        for page_number in range(number_of_pages):   # use xrange in Py2
            page = read_pdf.getPage(page_number)
            page_content = page.extractText()
            listi.append(page_content)
    return listi

### Penn Course Review (Ratings)

In [ ]:
# base url and API key
TOKEN = '' # insert token
base_url = 'http://api.penncoursereview.com/courses/'
token = '/reviews?token=' + TOKEN 

In [ ]:
# function to format term and course name for url

def format_course_url(syllabi):
    course_list = []
    for syllabus in syllabi:
        term= syllabus[39:44] #term
        dept= syllabus[45:49] #dept
        code= syllabus[49:52] #course code
        #section= syllabus[52:55] #section
    
        if term != '2019A':
            course_list.append(term.lower() + '-' + dept + '-' + code)
    
    return list(set(course_list))

In [ ]:
# get semester, course name, instructor name, ratings
final_rating_df = pd.DataFrame([])
error_course = []

for courses in course_list:

    json_file = requests.get(base_url + courses + token)
    
    if json_file.status_code == 200:
        df = pd.DataFrame()
        rating = pd.DataFrame()
        list_sections = json.loads(json_file.content.decode('utf-8')).get('result').get('values')
        term = courses[:5]

        for i in range(len(list_sections)):
            df = df.append([[term, 
                             list_sections[i].get('section').get('primary_alias'),
                             list_sections[i].get('instructor').get('name').title(),
                             list_sections[i].get('num_reviewers'),
                             list_sections[i].get('num_students')]])
            rating = rating.append(pd.DataFrame(list_sections[i].get('ratings'), index=[0]))
    
        df = df.reset_index(drop=True)
        df = df.rename(columns={0: 'term', 1:'course', 2:'instructor', 3:'num_reviewers', 4:'num_students'})

        rating = rating.reset_index(drop=True)
        df = pd.concat([df, rating], axis=1)
    
        final_rating_df = final_rating_df.append(df)
    
    else:
        error_course.append(courses)

In [ ]:
final_rating_df = final_rating_df.sort_values(by=['term', 'course']).reset_index(drop=True)

# we're only analyzing 2010 onwards
rating_df_2010 = final_rating_df[final_rating_df['term'] > '2010']

In [ ]:
def replace_class_name(x):
    return str.replace(x, "-","")

vect_name = np.vectorize(replace_class_name)
final_rating_2010['course'] = vect_name(final_rating_2010['course'])

### Penn Archives (Time and Location)

In [ ]:
# format of url is https://www.registrar.upenn.edu/archives/18B-Course-Timetable.pdf
first_url = 'https://www.registrar.upenn.edu/archives/'
second_url = '-Course-Timetable.pdf'

In [ ]:
# get course time and location data for 2010-2018A
terms = ['10A', '10C', '11A', '11C', '12A', '12C', '13A', '13B', '13C', '14A', '14B', '14C', '15A', '15B', '15C',
        '16A', '16B', '16C', '17A', '17B', '17C', '18A']
time_location_df = pd.DataFrame([])

for term in terms:
    # get txt of registrar data
    time_table = first_url + term + second_url
    formatted_output = ''.join(pdf_to_string(time_table)).replace('\n', '')
    
    # regex relevant info
    regex_all = re.findall('(?:(?:(?:ACCT|BEPP|MKTG|OIDD|FNCE|STAT|LGST|MGMT|HCMG|REAL)\-[0-9]{3})\s\s[A-Z])|' + 
                       '(?:[0-9]{3}[A-Z0-9 \n-:]{10,40}(?:VANC|JMHH|SHDH)\s[A-Z0-9]{3})', 
                       formatted_output)
    
    # clean regex-ed info
    regex_list = []
    for i in regex_all:
        match = re.match(r'[A-Z]{4}-[0-9]{3}', i)
        if match:
            regex_list.append(match.group(0))
        else:
            regex_list.append(i)
            
    # create dictionary
    test_dict = dict()
    for i in regex_list:
        if len(i) == 8:
            test_dict[key] = times
            times = []
            key = i
        else:
            if len(i) > 35:
                times.append(key + '-' + re.findall(r'(?:[0-9]{3}[A-Z0-9 -:)]{10,25}(?:VANC|JMHH|SHDH)\s[A-Z0-9]{3})',i)[0])
            else:
                times.append(key + '-' + i)
                
    # create dataframe
    flattened_list = [y for x in test_dict.values() for y in x]
    flattened_list = [i.split() for i in flattened_list]
    df = pd.DataFrame(flattened_list).iloc[:,0:6].rename(columns={0:'course', 1:'type', 2:'day', 3:'time', 4:'building', 5:'room'})
    df.insert(0, 'term', '20' + term.lower())
    
    # append df
    time_location_df = time_location_df.append(df)

time_location_df.reset_index(drop=True)

### Clean time_location_df

In [ ]:
# function to remove hyphens from courses
def remove_hyphen(string):
    return string.replace('-','')

In [ ]:
# remove hyphens from courses
time_location_df['course'] = time_location_df['course'].apply(remove_hyphen)
time_location_df.head(2)

In [ ]:
# function to get AM, PM or NOON
time_indicator = re.compile(".*\\-.*[0-9]([A-Z].*)")
def time_seperator(x):
    match = re.findall(time_indicator, x)
    try:
        return match[0]
    except:
        return ""

vect_time = np.vectorize(time_seperator)

In [ ]:
# function to split time into start and end
number = re.compile("(.*[0-9])[A-Z]*$")
def split_string(y):
    try:
        z = re.findall(number, y)[0]
        x = z.split("-")
        return x[0], x[1]
    except:
        return "",""

vect_split = np.vectorize(split_string)

In [ ]:
# create time indicator (AM, PM, NOON) column
time= vect_time(time_location_df['time'])
time_location_df["time_indicator"] = time

# create start time and end time column
time1, time2 = vect_split(time_location_df['time'])
time_location_df["start_time"] = time1
time_location_df["end_time"] = time2

In [ ]:
# function to convert time to decimal
def replace_half(x):
    if x.find(":")>0:
        minute_match = x.split(":")[1]
        minute = int(minute_match)
        frac = str(minute/60)
        minute_fin = ":" + str(minute_match)
        frac = '.' + frac.split('.')[1]
        return x.replace(minute_fin,frac)
    else:
        return x
    
vect_replace = np.vectorize(replace_half)

In [ ]:
# convert time to decimal
time_location_df['start_time'] = vect_replace(time_location_df["start_time"])
time_location_df['end_time'] = vect_replace(time_location_df["end_time"])

In [ ]:
# function to convert int to float
def to_float(x):
    z = float(x)
    return z

In [ ]:
# convert start time to float
start_time_float = []
for i in time_location_df['start_time']:
    try:
        start_time_float.append(to_float(i))
    except:
        start_time_float.append(to_float(100))

In [ ]:
# convert end time to float
end_time_float = []
for i in time_location_df['end_time']:
    try:
        end_time_float.append(to_float(i))
    except:
        end_time_float.append(to_float(100))

In [ ]:
# format existing dataframe
time_location_df['start_time'] = start_time_float
time_location_df['end_time'] = end_time_float
time_location_df.loc[time_location_df["time_indicator"]=="PM",'end_time'] = time_location_df.loc[time_location_df["time_indicator"]=="PM",'end_time'] + 12
time_location_df.loc[(time_location_df["time_indicator"]=="PM") & (time_location_df["start_time"]<8.0),'start_time'] = time_location_df.loc[(time_location_df["time_indicator"]=="PM") & (time_location_df["start_time"]<8.0),'start_time'] + 12 
time_location_df["class_duration"] = time_location_df["end_time"] - time_location_df["start_time"]

### Merge joined with syllabus

In [ ]:
# import joined data
joined_df = pd.read_csv('joined.csv').drop(['Unnamed: 0', 'day', 'time', 'building', 'room', 'instructor'], axis=1)
print(len(joined_df))
joined_df.head()

In [ ]:
# import syllabus data
syllabus = pd.read_csv('syllabus_complete.csv').drop('Unnamed: 0', axis=1)
syllabus = syllabus.rename(columns = {'class_name':'course', 'terms':'term', '0': 'url'})
syllabus.head(2)

In [ ]:
# function to add hyphen to course
def add_hyphen(course):
    return course[:4] + '-' + course[4:7] + '-' + course[7:]

add_hyphen_vect = np.vectorize(add_hyphen)

In [ ]:
# add hyphen to course
syllabus['course'] = add_hyphen_vect(syllabus['course'])

In [ ]:
# merge df to get final df
joined_df = joined_df.merge(syllabus, how='inner', on=['course','term'])
joined_df = joined_df.merge(time_location_df, how='inner', on=['course','term'])
joined_df = pd.concat([joined_df,pd.get_dummies(joined_df['day'])], axis=1)
joined_df = pd.concat([joined_df,pd.get_dummies(joined_df['building'])], axis=1)
len(joined_df)

In [ ]:
joined = joined_df.copy().dropna(subset=['test_percent', 'race', 'gender'])
joined["rCourseQualityBinned"] = joined["rCourseQuality"].astype("float").astype("int").astype('str')
joined['race'] = joined['race'].astype("float").astype("int").astype('str')
joined['gender'] = joined['gender'].astype("float").astype("int").astype('str')

# one hot encode gender
joined['gender'] = encode_gender_vect(joined['gender'])
joined = pd.concat([joined,pd.get_dummies(joined['gender'])], axis=1)

# one hot encode race
joined['race'] = encode_race_vect(joined['race'])
joined = pd.concat([joined,pd.get_dummies(joined['race'])], axis=1)

In [ ]:
# save to csv
# joined.to_csv('joined_final.csv', index=False)

### Clean final_rating_df because some courses used grad-level listing

In [ ]:
# function to check if course is grad-level listing
def check_if_grad_course(course):
    return int(re.findall('[0-9]{3}', course)[0]) > 500

check_if_grad_vect = np.vectorize(check_if_grad_course)

In [ ]:
index = np.where(check_if_grad_vect(final_rating_df['course'].unique()))
final_rating_df['course'].unique()[index]

In [ ]:
# function to change grad listing to undergrad listing
def change_to_undergrad(course):
    if course == 'STAT-512-401':
        return 'STAT-432-401'
    if course == 'STAT-510-401':
        return 'STAT-430-401'
    if course == 'STAT-701-401':
        return 'STAT-471-401'
    if course == 'STAT-701-402':
        return 'STAT-471-402'
    else:
        return course
    
change_to_undergrad_vect = np.vectorize(change_to_undergrad)

In [ ]:
# change grad listing to undergrad listing
final_rating_df['course'] = change_to_undergrad_vect(final_rating_df['course'])
final_rating_2010 = final_rating_df[final_rating_df['term'] > '2010']

## Dataframe Summaries

In [ ]:
pd.DataFrame(pdf_syllabus).head(5)

In [ ]:
final_rating_2010.head(5)

In [ ]:
time_location_df.head(5)